### 📘 Video 6: Build a Personal AI Assistant (LangChain for Beginners, Part 6)

**Goal**: Combine memory, tools, and smart decisions into one powerful AI assistant.

✅ No API keys

✅ Uses Hugging Face + LangChain

✅ Runs on free Colab GPU

🚀 You're now building real AI apps — just like Doug does at Illustris!

In [1]:
# Install only what we need
!pip install -q transformers torch accelerate bitsandbytes duckduckgo-search pdfplumber

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype="auto",
    trust_remote_code=True
)

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=150,
    temperature=0.7,
    do_sample=True,
    pad_token_id=tokenizer.eos_token_id
)

def llm(prompt):
    return pipe(prompt)[0]['generated_text'].replace(prompt, '').strip()

`torch_dtype` is deprecated! Use `dtype` instead!
Device set to use cpu


### 🛠️ Define Tools

In [4]:
from duckduckgo_search import DDGS

def search_tool(query):
    with DDGS() as ddgs:
        results = ddgs.text(query, max_results=3)
    return "\n".join([r['body'] for r in results])

def calculator_tool(expr):
    try:
        return str(eval(expr.replace(' ', '')))
    except:
        return "Error: Invalid expression"

### 🧠 Manual Memory (Simple & Reliable)

In [5]:
# Store conversation history manually
chat_history = []

def add_message(role, content):
    chat_history.append(f"{role}: {content}")

def get_context(window=4):
    return "\n".join(chat_history[-window:])

### 🧠 Agent Logic: Decide → Act

In [6]:
def ask(question):
    context = get_context()

    # Prompt with memory and decision logic
    prompt = f"<|im_start|>system\nYou are a helpful assistant.\nIf the question needs current data, reply 'USE_SEARCH'.\nIf it needs math, reply 'USE_CALC'.\nOtherwise, answer directly.<|im_end|>\n{context}\n<|im_start|>user\n{question}<|im_end|>\n<|im_start|>assistant\n"
    response = llm(prompt).lower()

    if "use_search" in response:
        print("🔍 Searching...")
        context = search_tool(question)[:1000]
        final_prompt = f"<|im_start|>system\nAnswer using only:\n{context}<|im_end|>\n{get_context()}\n<|im_start|>user\n{question}<|im_end|>\n<|im_start|>assistant\n"
        answer = llm(final_prompt)
        add_message("user", question)
        add_message("assistant", answer)
        return answer

    elif "use_calc" in response:
        print("🧮 Calculating...")
        expr = question.replace('what is', '').replace('?', '').strip()
        answer = calculator_tool(expr)
        add_message("user", question)
        add_message("assistant", answer)
        return answer

    else:
        answer = response.replace('use_search', '').replace('use_calc', '').strip()
        add_message("user", question)
        add_message("assistant", answer)
        return answer

### 🧪 Test Your AI Assistant

In [ ]:
print("🎙️  Welcome to Your Personal AI Assistant!\n")

print(f"🧑‍💻 You: My name is Alex. I like hiking.")
print(f"🤖 AI: {ask('My name is Alex. I like hiking.')}")
print()

print(f"🧑‍💻 You: What's my name?")
print(f"🤖 AI: {ask('What\'s my name?')}")
print()

print(f"🧑‍💻 You: What is 15 * 25?")
print(f"🤖 AI: {ask('What is 15 * 25?')}")
print()

print(f"🧑‍💻 You: Who won the 2024 Eurovision Song Contest?")
print(f"🤖 AI: {ask('Who won the 2024 Eurovision Song Contest?')}")

🎙️  Welcome to Your Personal AI Assistant!

🧑‍💻 You: My name is Alex. I like hiking.


### 🎉 Final Summary

You've built a full AI assistant with:

✅ **Memory** – remembers conversations

✅ **Tools** – searches the web, calculates

✅ **Agent logic** – decides when to act

💡 This is the foundation of **enterprise AI systems** — taught simply, freely, and powerfully

➡️ **Want more?**

📚 Enroll in Doug’s course: [Leveraging RAG with PostgreSQL](https://www.illustris.org/courses)

🎥 Subscribe: [YouTube @techbits-do](https://www.youtube.com/@techbits-do)

🔗 Connect: [LinkedIn /doug-ortiz-illustris](https://www.linkedin.com/in/doug-ortiz-illustris/)

💾 Code: [github.com/illustris-admin/ai/tree/main/langchain-for-beginners](https://github.com/illustris-admin/ai/tree/main/langchain-for-beginners)

Thank you for learning with Doug Ortiz — let's build impactful AI together!